In [ ]:
# Install necessary packages
!pip install torch transformers scikit-learn pandas numpy biopython peft bitsandbytes requests optuna

In [ ]:
# Install necessary tools in Google Colab
!apt-get install -y hmmer
!wget https://github.com/soedinglab/MMseqs2/releases/download/17-b804f/mmseqs-linux-gpu.tar.gz
!tar xvf mmseqs-linux-gpu.tar.gz
!chmod +x mmseqs

In [ ]:
import pandas as pd
import torch
import requests
from transformers import AutoTokenizer, AutoModelForMaskedLM, Trainer, TrainingArguments
from peft import LoraConfig, get_peft_model
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import Ridge, Lasso, ElasticNet
from sklearn.metrics import mean_squared_error
import numpy as np
from Bio import SeqIO
import optuna
from torch.utils.data import Dataset, DataLoader
import time
import xml.etree.ElementTree as ET
from scipy.stats import pearsonr
import matplotlib.pyplot as plt
import subprocess


# Check for GPU availability (Colab T4 GPU)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# URLs for external tools
BLAST_URL = "https://blast.ncbi.nlm.nih.gov/Blast.cgi"

# Wild-type enzyme sequence
WT_SEQUENCE = "MRHGDISSSNDTVGVAVVNYKMPRLHTAAEVLDNARKIAEMIVGMKQGLPGMDLVVFPEYSLQGIMYDPAEMMETAVAIPGEETEIFSRACRKANVWGVFSLTGERHEEHPRKAPYNTLVLIDNNGEIVQKYRKIIPWCPIEGWYPGGQTYVSEGPKGMKISLIICDDGNYPEIWRDCAMKGAELIVRCQGYMYPAKDQQVMMAKAMAWANNCYVAVANAAGFDGVYSYFGHSAIIGFDGRTLGECGEEEMGIQYAQLSLSQIRDARANDQSQNHLFKILHRGYSGLQASGDGDRGLAECPFEFYRTWVTDAEKARENVERLTRSTTGVAQCPVGRLPYEG"


def run_blast_search(wt_sequence, identity_threshold=98.0, max_retries=50, sleep_time=20):
    """Run BLAST search and extract homologous sequences below a given identity threshold."""

    # **1. Submit BLAST Query**
    params = {
        "CMD": "Put",
        "PROGRAM": "blastp",
        "DATABASE": "nr",
        "QUERY": wt_sequence,
        "FORMAT_TYPE": "XML",
        "EXPECT": "10",  # Set e-value threshold (adjustable)
        "HITLIST_SIZE": "500"  # Increase number of results
    }

    response = requests.post(BLAST_URL, data=params)
    if response.status_code != 200:
        raise Exception(f"Error submitting sequence: {response.text}")

    response_text = response.text
    if "RID = " not in response_text:
        raise Exception("Error: No RID found in response.")

    rid = response_text.split("RID = ")[-1].split("\n")[0].strip()
    print(f"🔵 Submitted BLAST job with RID: {rid}")

    # **2. Wait for BLAST results**
    retries = 0
    while retries < max_retries:
        status_params = {"CMD": "Get", "FORMAT_OBJECT": "SearchInfo", "RID": rid}
        status_response = requests.get(BLAST_URL, params=status_params)

        if "Status=READY" in status_response.text:
            print("🟢 BLAST search is complete.")
            break
        elif "Status=FAILED" in status_response.text:
            raise Exception("🔴 BLAST job failed on NCBI server.")

        print(f"⏳ Waiting for BLAST results... (Attempt {retries + 1}/{max_retries})")
        time.sleep(sleep_time)
        retries += 1

    if retries == max_retries:
        raise Exception("🔴 BLAST API timed out. Try increasing max_retries or sleep_time.")

    # **3. Retrieve BLAST results**
    result_params = {"CMD": "Get", "FORMAT_TYPE": "XML", "RID": rid}
    result_response = requests.get(BLAST_URL, params=result_params)
    if result_response.status_code != 200:
        raise Exception(f"Error retrieving results: {result_response.text}")

    # Print raw response for debugging
    print(result_response.text[:1000])  # Print first 1000 characters to check if XML is valid

    # **4. Parse BLAST results**
    root = ET.fromstring(result_response.text)
    homologous_sequences = []
    query_sequences = []
    identities = []

    for hit in root.findall(".//Hit"):
        hit_id = hit.find("Hit_id").text if hit.find("Hit_id") is not None else "Unknown"
        hit_def = hit.find("Hit_def").text if hit.find("Hit_def") is not None else "Unknown"

        for hsp in hit.findall(".//Hsp"):
            hsp_qseq_elem = hsp.find("Hsp_qseq")
            hsp_hseq_elem = hsp.find("Hsp_hseq")
            identity_elem = hsp.find("Hsp_identity")
            align_len_elem = hsp.find("Hsp_align-len")

            if hsp_qseq_elem is None or hsp_hseq_elem is None or identity_elem is None or align_len_elem is None:
                continue  # Skip if missing data

            qseq = hsp_qseq_elem.text.strip()
            hseq = hsp_hseq_elem.text.strip()
            identity = int(identity_elem.text)
            align_len = int(align_len_elem.text)
            identity_percentage = (identity / align_len) * 100

            if identity_percentage <= identity_threshold:
                homologous_sequences.append(hseq)  # Homologous sequences
                query_sequences.append(qseq)  # Query sequences
                identities.append(identity_percentage)
                print(f"✅ Found sequence from {hit_id}: {hit_def} (Identity: {identity_percentage:.2f}%)")

    print(f"✅ Total homologous sequences collected: {len(homologous_sequences)}")
    print(f"✅ Total query sequences collected: {len(query_sequences)}")

    return homologous_sequences, query_sequences, identities

# **Run BLAST API Search**
homologous_sequences, _, _ = run_blast_search(WT_SEQUENCE)


print(f"🔹 Retrieved {len(homologous_sequences)} homologous sequences.")
#print(f"🔹 Retrieved {len(query_sequences)} query sequences.")
#print(f"🔹 Average Sequence Identity: {sum(identities)/len(identities) if identities else 0:.2f}%")


def run_jackhmmer_search(wt_sequence, num_iter=3, evalue=1e-5):
    """Run Jackhmmer search using HMMER locally. Assumes HMMER is installed."""
    fasta_filename = "wt_sequence.fasta"
    with open(fasta_filename, "w") as f:
        f.write(f">WT_SEQUENCE\n{wt_sequence}\n")

    output_file = "jackhmmer_output.txt"
    cmd = f"jackhmmer --tblout {output_file} -N {num_iter} --cpu 4 -E {evalue} {fasta_filename} /usr/share/uniprot_sprot.fasta"

    subprocess.run(cmd, shell=True, check=True)

    sequences = []
    with open(output_file, "r") as f:
        for line in f:
            if not line.startswith("#"):
                columns = line.strip().split()
                if len(columns) > 1:
                    sequences.append(columns[0])  # Extract sequence ID

    print(f"Total sequences collected from Jackhmmer: {len(sequences)}")
    return sequences if sequences else [wt_sequence]


def run_mmseqs2_search(wt_sequence):
    """Run MMseqs2 locally in Google Colab."""

    # Save WT sequence to FASTA file
    fasta_filename = "wt_sequence.fasta"
    with open(fasta_filename, "w") as f:
        f.write(f">WT_SEQUENCE\n{wt_sequence}\n")

    db_name = "uniref50"
    output_file = "mmseqs_output.m8"

    # Create database, search, and convert output
    subprocess.run(f"mmseqs createdb {fasta_filename} wt_sequence_db", shell=True, check=True)
    subprocess.run(f"mmseqs search wt_sequence_db {db_name} results tmp --threads 4", shell=True, check=True)
    subprocess.run(f"mmseqs convertalis wt_sequence_db {db_name} results {output_file}", shell=True, check=True)

    sequences = []
    with open(output_file, "r") as f:
        for line in f:
            columns = line.strip().split("\t")
            if len(columns) > 1:
                sequences.append(columns[1])  # Extract sequence ID

    print(f"Total sequences collected from MMseqs2: {len(sequences)}")
    return sequences if sequences else [wt_sequence]


# Example Usage
method = "blast"  # Change to "jackhmmer" or "mmseqs2" as needed

if method == "blast":
    homologous_sequences, _, _ = run_blast_search(WT_SEQUENCE)
elif method == "jackhmmer":
    homologous_sequences = run_jackhmmer_search(WT_SEQUENCE)
elif method == "mmseqs2":
    homologous_sequences = run_mmseqs2_search(WT_SEQUENCE)
else:
    raise ValueError("Invalid method chosen. Please select 'blast', 'jackhmmer', or 'mmseqs2'.")

print(f"Retrieved {len(homologous_sequences)} sequences using {method}.")

# Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load ProtBERT Tokenizer and Model
print("Loading tokenizer and model...")
tokenizer = AutoTokenizer.from_pretrained("Rostlab/prot_bert")

# Avoid float16 for CPU memory safety
base_model = AutoModelForMaskedLM.from_pretrained("Rostlab/prot_bert").to(device)

# Embedding extractor

def get_protbert_embedding(model, sequence):
    if not isinstance(sequence, str) or len(sequence) == 0:
        raise ValueError("Invalid input: `sequence` must be a non-empty string.")
    with torch.no_grad():
        tokens = tokenizer(sequence, return_tensors="pt", padding=True, truncation=True, max_length=512).to(device)
        outputs = model(**tokens, output_hidden_states=True)
        if outputs.hidden_states:
            last_hidden_state = outputs.hidden_states[-1]
            embedding = last_hidden_state.mean(dim=1).squeeze().cpu().numpy()
            return embedding
        else:
            raise AttributeError("Model outputs do not contain 'hidden_states'.")

# Mutation utility

def generate_mutated_sequence(wt_sequence, mutation_string):
    seq_list = list(wt_sequence)
    mutations = mutation_string.split(',')
    for mut in mutations:
        if len(mut) >= 3 and mut[1:-1].isdigit():
            pos = int(mut[1:-1]) - 1
            if pos < len(seq_list) and mut[-1] != '*':
                seq_list[pos] = mut[-1]
    return ''.join(seq_list)

# Load data
print("Loading dataset...")
url = "https://figshare.com/ndownloader/files/7337543"
df = pd.read_csv(url, sep='\t')
df.rename(columns={'mutation': 'mutation_string', 'normalized_fitness': 'fitness'}, inplace=True)
df['fitness'] = pd.to_numeric(df['fitness'], errors='coerce')
df.dropna(subset=['fitness'], inplace=True)
df = df.sample(20, random_state=1)  # Reduce for memory safety

# Wildtype
WT_SEQUENCE = "MSKGEELFTGVVPILVELDGDVNGHKFSVSGEGEGDATYGKLTLKFICTTGKLPVPWPTLVTTLTYGVQCFSRYPDHMKQHDFFKSAMPEGYVQERTIFFKDDGNYKTRAEVKFEGDTLVNRIELKGIDFKEDGNILGHKLEYNYNSHNVYIMADKQKNGIKVNFKIRHNIEDGSVQLADHYQQNTPIGDGPVLLPDNHYLSTQSALSKDPNEKRDHMVLLEFVTAAGITHGMDELYK"
df['mutated_sequence'] = df['mutation_string'].apply(lambda x: generate_mutated_sequence(WT_SEQUENCE, x))

# Generate embeddings
print("Generating embeddings...")
embeddings = []
y = []
for i, row in df.iterrows():
    try:
        emb = get_protbert_embedding(base_model, row['mutated_sequence'])
        embeddings.append(emb)
        y.append(row['fitness'])
    except Exception as e:
        print(f"Skipping sequence due to error: {e}")

X = np.vstack(embeddings)
y = np.array(y)

# Scale
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# 👇 This is key: get the indices first
all_indices = np.arange(len(X_scaled))
X_train_index, X_test_index = train_test_split(all_indices, test_size=0.2, random_state=42)

# Get the actual data from the indices
X_train = X_scaled[X_train_index]
X_test = X_scaled[X_test_index]
y_train = y[X_train_index]
y_test = y[X_test_index]

# Dummy train_dataloader for this example (no LoRA training actually done here)
train_dataloader = None  # Replace with real one in full setup

# Objective function for Optuna
def objective(trial, model_name, dataloader, X_test, y_test):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    model = AutoModelForMaskedLM.from_pretrained(model_name).to(device)

    r = trial.suggest_int("r", 4, 64, step=4)
    alpha = trial.suggest_int("alpha", 8, 128, step=8)
    lora_config = LoraConfig(
        r=r,
        lora_alpha=alpha,
        target_modules=["query", "value"],
        lora_dropout=0.1
    )

    model = get_peft_model(model, lora_config).to(device)

    # Inference without training — use mean pooling to reduce embedding to scalar
    y_pred = [np.mean(get_protbert_embedding(model, seq)) for seq in X_test]

    y_pred = np.array(y_pred)
    y_test = np.array(y_test)
    assert y_pred.shape == y_test.shape, f"Shape mismatch: {y_pred.shape} vs {y_test.shape}"

    return mean_squared_error(y_test, y_pred)



# Run Optuna (1 trial for safety)
print("Running Optuna study...")
study = optuna.create_study(direction="minimize")
study.optimize(lambda trial: objective(
    trial,
    "Rostlab/prot_bert",
    train_dataloader,
    df.iloc[X_test_index]['mutated_sequence'].tolist(),
    df.iloc[X_test_index]['fitness'].values), n_trials=20)

print("Best params:", study.best_params)


# Custom Dataset Class for Fine-tuning
class ProteinDataset(Dataset):
    def __init__(self, sequences, tokenizer, max_length=512):
        self.sequences = sequences
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.sequences)

    def __getitem__(self, idx):
        sequence = self.sequences[idx]
        inputs = self.tokenizer(sequence, truncation=True, padding='max_length', max_length=self.max_length, return_tensors="pt")
        return {
            "input_ids": inputs["input_ids"].squeeze(0),
            "attention_mask": inputs["attention_mask"].squeeze(0),
        }

# Fine-tune ProtBERT
def train_protbert(model, dataloader):
    if dataloader is None or not isinstance(dataloader, DataLoader):
        print("Skipping training: No valid dataloader found for fine-tuning.")
        return

    model.train()
    optimizer = torch.optim.AdamW(model.parameters(), lr=3e-5, eps=1e-8)

    for epoch in range(1):
        for batch in dataloader:
            batch = {k: v.to(device) for k, v in batch.items()}

            input_ids = batch["input_ids"]
            attention_mask = batch["attention_mask"]
            labels = input_ids.clone()

            outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
            if outputs.loss is None:
                print("⚠️ Warning: Model returned no loss. Skipping batch.")
                continue

            loss = outputs.loss
            loss.backward()
            optimizer.step()
            optimizer.zero_grad()

        print(f"✅ Epoch {epoch+1} completed")

def objective(trial, model_name, dataloader, X_test, y_test):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    model = AutoModelForMaskedLM.from_pretrained(model_name).to(device)

    r = trial.suggest_int("r", 2, 16, step=2)
    alpha = trial.suggest_int("alpha", 4, 32, step=4)
    lora_config = LoraConfig(
        r=r,
        lora_alpha=alpha,
        target_modules=["query", "value"],
        lora_dropout=0.1
    )

    model = get_peft_model(model, lora_config).to(device)

    # Debug: print shape of a sample embedding
    sample_emb = get_protbert_embedding(model, X_test[0])
    print("Embedding shape (sample):", np.array(sample_emb).shape)

    # Convert embeddings to scalar using mean pooling
    y_pred = [np.mean(np.array(get_protbert_embedding(model, seq)).squeeze()) for seq in X_test]

    y_pred = np.array(y_pred)
    y_test = np.array(y_test)
    assert y_pred.shape == y_test.shape, f"Shape mismatch: {y_pred.shape} vs {y_test.shape}"

    return mean_squared_error(y_test, y_pred)



# Prepare dataset and dataloader
if not 'homologous_sequences' in globals() or not isinstance(homologous_sequences, list) or len(homologous_sequences) == 0:
    raise ValueError("Error: No sequences available for training.")

train_dataset = ProteinDataset(homologous_sequences, tokenizer)
train_dataloader = DataLoader(train_dataset, batch_size=1, shuffle=True)

# Run Optuna Optimization
study = optuna.create_study(direction="minimize")
study.optimize(lambda trial: objective(
    trial,
    "Rostlab/prot_bert",
    train_dataloader,
    df.iloc[X_test_index]['mutated_sequence'].tolist(),
    df.iloc[X_test_index]['fitness'].values), n_trials=20)


# Apply best LoRA parameters
best_lora_params = study.best_params
print(f"🏆 Best LoRA Parameters (Optuna): {best_lora_params}")

config = LoraConfig(
    r=best_lora_params["r"],
    lora_alpha=best_lora_params["alpha"],
    target_modules=["query", "value"],
    lora_dropout=0.1
)

model = AutoModelForMaskedLM.from_pretrained("Rostlab/prot_bert").to(device)
model = get_peft_model(model, config)
model.print_trainable_parameters()

# Final Fine-tuning
train_protbert(model, train_dataloader)

# Function to train, evaluate, and plot results
def train_and_evaluate_model(model, param_grid, X_train, y_train, X_test, y_test, model_name):
    cv_scores = cross_val_score(model, X_train, y_train, cv=5, scoring='neg_mean_squared_error')
    print(f"{model_name} - Cross-validation MSE (mean): {-cv_scores.mean():.4f}")

    grid_search = GridSearchCV(model, param_grid, cv=5, scoring='neg_mean_squared_error')
    grid_search.fit(X_train, y_train)

    print(f"Best parameters for {model_name}: {grid_search.best_params_}")

    y_pred = grid_search.predict(X_test)

    mse = mean_squared_error(y_test, y_pred)
    print(f"Mean Squared Error (MSE) - {model_name}: {mse:.4f}")

    pearson_corr, _ = pearsonr(y_test, y_pred)
    print(f"Pearson Correlation Coefficient - {model_name}: {pearson_corr:.4f}")

    plt.figure(figsize=(8, 6))
    plt.scatter(y_test, y_pred, color='blue', alpha=0.6)
    plt.plot([min(y_test), max(y_test)], [min(y_test), max(y_test)], color='red', linestyle='--')
    plt.xlabel('Actual Fitness')
    plt.ylabel('Predicted Fitness')
    plt.title(f'Actual vs Predicted Fitness ({model_name})')
    plt.grid(True)
    plt.show()

    return grid_search

# Define parameter grids for each model
param_grid_ridge = {'alpha': [0.01, 0.1, 1, 10, 100]}
param_grid_lasso = {'alpha': [0.01, 0.1, 1, 10, 100]}
param_grid_en = {'alpha': [0.01, 0.1, 1, 10, 100], 'l1_ratio': [0.1, 0.5, 0.9, 1.0]}

# Define models
ridge_model = Ridge()
lasso_model = Lasso()
elastic_net_model = ElasticNet()

# Train and evaluate each model
train_and_evaluate_model(ridge_model, param_grid_ridge, X_train, y_train, X_test, y_test, "Ridge")
train_and_evaluate_model(lasso_model, param_grid_lasso, X_train, y_train, X_test, y_test, "Lasso")
train_and_evaluate_model(elastic_net_model, param_grid_en, X_train, y_train, X_test, y_test, "ElasticNet")
